# Kinopoisk film parser

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import requests    
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from random import randrange
import time 
import socks
import socket
import resource
import sys
import re
import pickle
import itertools


import warnings
warnings.filterwarnings('ignore')


In [2]:
base_url = "http://kinopoisk.ru/top/navigator/m_act[ecountry]/20/m_act[rating]/1%3A/" 
# for search should be at least one filter, this one exludes Algeria and search all films with rating from 1 to 10
order_by = {"kinopoisk": "/order/rating", "imdb":"/order/ex_rating", "year":"/order/year"}

In [3]:
def get_link(order = "imdb", page = 1):
    return base_url + order_by[order] + "/page/{}/#results".format(page)

In [14]:
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [4]:
def get_soup(link, agent = UserAgent().firefox, verbose=False):
    if verbose:
        print("Sending request: ", link)
    response = requests.get(link, headers={'User-Agent': agent})
    if not response.ok:
        return []
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/lib/python3.8/urllib/request.py", line 1350, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/usr/lib/python3.8/http/client.py", line 1255, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.8/http/client.py", line 1301, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.8/http/client.py", line 1250, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.8/http/client.py", line 1010, in _send_output
    self.send(msg)
  File "/usr/lib/python3.8/http/client.py", line 950, in send
    self.connect()
  File "/usr/lib/python3.8/http/client.py", line 921, in connect
    self.sock = self._create_connection(
  File "/usr/lib/python3.8/socket.py", l

In [7]:
soups = []
page_count = 510
block_count = 0
for i in tqdm_notebook(range(page_count)):
    time.sleep(randrange(5))
    soup = get_soup(get_link(page=i), agent = UserAgent().random)
    if 'captcha' in str(soup):
        for j in range(3):
            time.sleep(randrange(7))
            soup = get_soup(get_link(page=i), agent = UserAgent().random)
    if 'captcha' in str(soup):
        block_count = block_count + 1
        soup = None
        print("All request was blocked for page: ", i)
        time.sleep(20)
    soups.append(soup)

All request was blocked for page:  340
All request was blocked for page:  349
All request was blocked for page:  350
All request was blocked for page:  362
All request was blocked for page:  363
All request was blocked for page:  378
All request was blocked for page:  381
All request was blocked for page:  386
All request was blocked for page:  389
All request was blocked for page:  391
All request was blocked for page:  394
All request was blocked for page:  395
All request was blocked for page:  396
All request was blocked for page:  400
All request was blocked for page:  401
All request was blocked for page:  403
All request was blocked for page:  405
All request was blocked for page:  406
All request was blocked for page:  407
All request was blocked for page:  408
All request was blocked for page:  416
All request was blocked for page:  426
All request was blocked for page:  437
All request was blocked for page:  459
All request was blocked for page:  471
All request was blocked f

In [5]:
print (sys.getrecursionlimit())
sys.setrecursionlimit(50000)

3000


In [10]:
file_path = 'file_path'

with open(file_path, 'wb') as f:
    pickle.dump(soups, f)

In [19]:
filtered = [x for x in soups if x is not None]
len(filtered)

424

### Мы собрали информацию примерно 80% страниц фильмов, думаю должно быть достаточно. Теперь надо вытащить идентификаторы фильмов.

In [42]:
film_ids = []
for soup in soups:
    if soup is None:
        continue 
    divs = soup.find_all('div', attrs={'class': "name"})
    for div in divs:
        film_ids.append(re.search('.*/film/(.*)/',div.a['href']).group(1))
        
film_ids[0:10:1][0]

'5127'

In [6]:
def get_film_link(id):
    return 'http://kinopoisk.ru/film/{}'.format(id)

In [12]:
file_path = '/home/mikhail/PycharmProjects/intro/models/fileids.pickle'

with open(file_path, 'rb') as f:
    film_ids = pickle.load(f)

In [10]:
get_soup(get_film_link(film_ids[0]), agent = UserAgent().random)

<!DOCTYPE html>
<html><head><script data-tid="23a8082c">!function(i,t){if(i.Ya=i.Ya||{},Ya.Rum)throw new Error("Rum: interface is already defined");var n=i.performance,e=n&&n.timing&&n.timing.navigationStart||Ya.startPageLoad||+new Date,s=i.requestAnimationFrame;Ya.Rum={enabled:!!n,vsStart:document.visibilityState,vsChanged:!1,_defTimes:[],_defRes:[],_deltaMarks:{},_markListeners:{},_settings:{},_vars:{},init:function(i,t){this._settings=i,this._vars=t},getTime:n&&n.now?function(){return n.now()}:Date.now?function(){return Date.now()-e}:function(){return new Date-e},time:function(i){this._deltaMarks[i]=[this.getTime()]},timeEnd:function(i,t){var n=this._deltaMarks[i];n&&0!==n.length&&n.push(this.getTime(),t)},sendTimeMark:function(i,t,n,e){void 0===t&&(t=this.getTime()),this._defTimes.push([i,t,e]),this.mark(i,t)},sendResTiming:function(i,t){this._defRes.push([i,t])},sendRaf:function(i){var t=this.getSetting("forceFirstPaintTimeSending");if(s&&(t||!this.isVisibilityChanged())){var n=th

In [7]:
def save_films(films, num_of_iter):
    file_path = '/home/mikhail/PycharmProjects/intro/models/filmsoups{}.pickle'.format(num_of_iter)
    with open(file_path, "wb") as f:
        pickle.dump(films, f)

In [8]:
def get_films(num_of_iter):
    file_path = '/home/mikhail/PycharmProjects/intro/models/filmsoups{}.pickle'.format(num_of_iter)
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [13]:
film_soups = []
not_received = []
count = itertools.count(32)
dump_start = 0
dump_freq = 500

for film_id in tqdm_notebook(film_ids[15500:]):
    try:
        time.sleep(1)
        film_soup = get_soup(get_film_link(film_id), agent = UserAgent().random)
        film_soups.append(film_soup)
        current_iter = next(count)
        if  current_iter % dump_freq == 0:
            print("Saving pack of films with number: {}", current_iter // dump_freq)
            save_films(film_soups[dump_start:], current_iter // dump_freq)
    except:
        print("Ooops!")

Saving pack of films with number: {} 1
Saving pack of films with number: {} 2
Saving pack of films with number: {} 3
Saving pack of films with number: {} 4
Saving pack of films with number: {} 5
Ooops!
Saving pack of films with number: {} 6
Saving pack of films with number: {} 7
Saving pack of films with number: {} 8
Saving pack of films with number: {} 9
Saving pack of films with number: {} 10
Saving pack of films with number: {} 11



In [ ]:
films = len([x for x in film_soups if 'captcha' not in str(x)])

In [10]:
films = []
for i in tqdm_notebook(range(32)):
    films.extend(film for film in get_films(i) if 'captcha' not in str(film))
films = len([x for x in film_soups if 'captcha' not in str(x)])

KeyboardInterrupt: 

In [102]:
def find_basic_info(soup, text, attr_name, film):
    tmp = soup.find('div',text=text).parent
    if tmp is not None:
        tmp = tmp.select_one('div:nth-child(2n) a')
        if tmp is not None:
            film[attr_name] = tmp.text.replace(u'\xa0', u'')

def parse(soup):
    film = {}
    film_id = soup.select_one("meta[property='og:url']")
    if film_id is not None:
        film['id'] = int(re.search('.*/film/(.*)/',film_id['content']).group(1))
    
    name = soup.select_one('h1', attrs={'class*':'styles_title'})
    if name is not None:
        film['ru_name'] = name.span.text
        
    origin_name = soup.select_one("span[class*='styles_originalTitle']")
    if origin_name is not None:
        film['origin_name'] = origin_name.text
        
    basic_info_div = soup.select_one("div[class*='basicInfo']")
    basic_info_tuples = [("Год производства", 'make_year'), ("Страна", "country"), ("Жанр", 'genres'),
                        ('Слоган', 'Tagline'), ('Режиссер','director'), ('Сценарий', 'scriptwriter'),
                        ('Продюсер','producer'), ('Оператор','operator'), ('Композитор','music'),
                        ('Художник','artist'), ('Монтаж', 'edit'),('Бюджет','cost'), ('Маркетинг','marketing'),
                        ('Сборы в США','usa_fees'), ('Сборы в мире','world_fees'), ('Премьера в Росcии','rus_premiere'),
                        ('Премьера в мире','world_premiere'), ('Релиз на DVD', 'dvd_release')]
    
    if basic_info_div is not None:
        for text, attr_name in basic_info_tuples:
            find_basic_info(soup, text, attr_name, film)
    
    synopsis = soup.select_one("div[class*='styles_synopsis'] p ")
    if synopsis is not None:
        film['synopsis'] = synopsis.text
    
    kinopoisk_rating = soup.select_one('a.film-rating-value')
    if kinopoisk_rating is not None:
        film['kinopoisk_rating'] = kinopoisk_rating.text
    
    imdb_rating = soup.select_one("div.film-sub-rating > span[class*='value']")
    if imdb_rating is not None:
         film['imdb_rating'] = imdb_rating.text
            
    actors = soup.select_one("div.film-crew-block").select("div.film-crew-block > div > ul > li > a")
    if actors is not None and len(actors) != 0:
        film['first_5_actors'] = ",".join(actor_soup.text for actor_soup in actors[:5])
    return film

In [103]:
pd.DataFrame([parse(soup)])

,id,ru_name,origin_name,make_year,country,genres,director,scriptwriter,producer,operator,...,cost,marketing,usa_fees,world_fees,rus_premiere,world_premiere,synopsis,kinopoisk_rating,imdb_rating,first_5_actors
0,5127,Недетское кино,Not Another Teen Movie,2001,США,комедия,Джоэл Галлен,Майк Бендер,Нил Х. Мориц,Рейнальдо Вильялобос,...,$15000000,$18000000,$38252284,+ $28216048 = $66468332,21 февраля 2002,7 декабря 2001,Популярный в школе парень и звезда футбольной ...,5.856,IMDb: 5.70,"Кайлер Ли,Крис Эванс,Джейми Прессли,Эрик Крист..."
